File 01-binary_netherlands.py


Michel Bierlaire

Thu Aug 8 09:55:47 2024






In [1]:

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from IPython.core.display_functions import display
from biogeme.expressions import Beta, Variable, log, exp


The goal of this computer session is to

1. become familiar with the Python syntax in Biogeme and
2. estimate and interpret a binary logit model.

We are using an old dataset for a binary transportation mode choice,
collected in the Netherlands. The data set is available as
http://transp-or.epfl.ch/data/netherlands.dat, and its description is
available as
http://transp-or.epfl.ch/documents/technicalReports/CS_NetherlandsDescription.pdf.

We recommend to download the dataset in your local directory.

# Data preparation

We first import the data into Pandas, using any interface that
Pandas allows. Here, we simply read the data from a text file, where
the data are separated by tabs.

In [2]:
df = pd.read_csv('netherlands.dat', sep='\t')
display(df)


,id,rp,sp,choice,purpose,npersons,age,employ_status,mainearn,arrival_time,...,seat_status,car_ivtt,car_cost,car_walk_time,car_parking_fee,rail_comfort,rp_transfer,rp_choice,rp_rail_ovt,rp_car_ovt
0,1,1,0,0,0,3,0,0,0,0,...,0,1.000,5.000,0.167,0,-1,0,0,0.50,0.167
1,1,0,1,10,0,3,0,0,0,0,...,-1,1.167,7.500,-1.000,-1,0,0,0,0.50,0.167
2,1,0,1,10,0,3,0,0,0,0,...,-1,1.167,7.000,-1.000,-1,0,0,0,0.50,0.167
3,1,0,1,10,0,3,0,0,0,0,...,-1,1.167,6.500,-1.000,-1,0,0,0,0.50,0.167
4,1,0,1,10,0,3,0,0,0,0,...,-1,1.167,6.000,-1.000,-1,0,0,0,0.50,0.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1734,235,0,1,10,0,4,1,1,0,1,...,-1,1.517,7.500,-1.000,-1,1,2,0,0.25,0.017
1735,235,0,1,10,0,4,1,1,0,1,...,-1,1.517,6.875,-1.000,-1,1,2,0,0.25,0.017
1736,235,0,1,10,0,4,1,1,0,1,...,-1,1.517,6.250,-1.000,-1,1,2,0,0.25,0.017
1737,235,0,1,10,0,4,1,1,0,1,...,-1,1.517,6.250,-1.000,-1,0,2,0,0.25,0.017


We then import this database into Biogeme.

In [3]:
database = db.Database('netherlands', df)


We identify the columns that will be used as variable in our model.

In [4]:
sp = Variable('sp')
rail_ivtt = Variable('rail_ivtt')
rail_acc_time = Variable('rail_acc_time')
rail_egr_time = Variable('rail_egr_time')
car_ivtt = Variable('car_ivtt')
car_walk_time = Variable('car_walk_time')
car_cost = Variable('car_cost')
rail_cost = Variable('rail_cost')
choice = Variable('choice')


The data set contains both stated preferences (SP) and revealed
preferences (RP) data. We are using only RP data. Therefore, we
exclude the SP data.

In [5]:
exclude = sp != 0
database.remove(exclude)


We can see that the data set has reduced from 1739 rows down to 228 rows.

In [6]:
print(f'Shape of the data: {database.data.shape}')


Shape of the data: (228, 28)


Here is the reduced dat set.

In [7]:
display(database.data)


,id,rp,sp,choice,purpose,npersons,age,employ_status,mainearn,arrival_time,...,seat_status,car_ivtt,car_cost,car_walk_time,car_parking_fee,rail_comfort,rp_transfer,rp_choice,rp_rail_ovt,rp_car_ovt
0,1,1,0,0,0,3,0,0,0,0,...,0,1.000,5.000,0.167,0,-1,0,0,0.500,0.167
9,2,1,0,0,0,5,0,1,0,0,...,0,1.500,9.000,0.017,1,-1,0,0,0.383,0.017
18,3,1,0,0,0,2,0,0,0,0,...,0,1.633,11.500,0.333,0,-1,0,0,0.517,0.333
25,4,1,0,0,0,3,1,1,1,0,...,0,1.500,8.333,0.500,1,-1,1,0,0.750,0.500
30,5,1,0,0,0,3,0,0,0,0,...,0,1.250,5.000,0.017,1,-1,1,0,0.367,0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1707,231,1,0,0,0,2,0,0,0,0,...,0,1.333,14.000,0.067,1,-1,1,0,0.750,0.067
1712,232,1,0,0,0,4,0,0,1,0,...,0,1.333,17.500,0.000,1,-1,1,0,0.667,0.000
1720,233,1,0,0,0,2,1,0,1,1,...,0,1.500,6.250,0.333,1,-1,0,0,0.333,0.333
1725,234,1,0,0,0,3,0,1,0,0,...,0,1.250,21.667,0.017,0,-1,1,0,0.500,0.017


We can aso define new variables from existing one.

The total travel time by rail is the sum of the in-vehicle travel
time, the access time (time from the origin of the trip to the first
train station) and the egress time (time from the last train station
to the final destination).

In [8]:
rail_time = rail_ivtt + rail_acc_time + rail_egr_time


The total travel time by car is the sum of the in-vehicle travel
time and the walking time, to and from the parking.

In [9]:
car_time = car_ivtt + car_walk_time


The data set has been collected before the existence of Euro, and
the costs are coded in Dutch Guilders. In order to simplify the
interpretation of the results, we use the conversion of Guilders
into Euros.

In [10]:
DUTCH_GUILDERS_TO_EUROS = 0.44378022
car_cost_euro = car_cost * DUTCH_GUILDERS_TO_EUROS
rail_cost_euro = rail_cost * DUTCH_GUILDERS_TO_EUROS


# Model specification

We are now ready to specify the choice model. We start with a simple
model, that contains only one alternative specific constant:
\begin{align*}
V_\text{car} &= \text{ASC}_\text{car}, \\ V_\text{rail} &= 0.
\end{align*}

We define the unknown parameter using the Biogeme expression `Beta`,
that takes 5 arguments:

- the name of the parameter (it is advised to use the exact same
name for the corresponding Python variable),
- the starting value for the estimation (usually, 0),
- a lower bound on the value of the coefficient, or `None` for no
bound,
- an upper bound, or `None`for no bound,
- a parameter that is 1 if the value of the parameter must be fixed
to its starting value, and 0 if it has to be estimated.

In [11]:
asc_car = Beta(name='asc_car', value=0, lowerbound=None, upperbound=None, status=0)


We now write the utility functions:

In [12]:
v_car = asc_car
v_rail = 0


And we write the choice model:

In [13]:
prob_car = 1 / (1 + exp(v_rail - v_car))
prob_rail = 1 - prob_car


Biogeme needs the formula of the contribution of each observation to
the log likelihood function, which depends on the observed choice:

In [14]:
prob_observation = prob_car * (choice == 0) + prob_rail * (choice == 1)
logprob = log(prob_observation)


We initialize Biogeme with this expression, and the database.

In [15]:
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'binary_netherlands'


# Estimation of the parameter(s)

We are now ready to estimate the parameter. Biogeme tries to read a
file `__binary_netherlands.iter` containing intermediary results
from a previous estimation run. If it does not find it, it triggers
a warning that can be ignored.

In [16]:
results = biogeme.estimate()


The results are stored in a object that allows to access various
information about the estimation. Use the help function to have a
detailed description.

In [17]:
help(results)


Help on bioResults in module biogeme.results object:

class bioResults(builtins.object)
 |  bioResults(the_raw_results: 'RawResults | None' = None, pickle_file: 'str | None' = None, identification_threshold: 'float | None' = None)
 |  
 |  Class managing the estimation results
 |  
 |  Methods defined here:
 |  
 |  __init__(self, the_raw_results: 'RawResults | None' = None, pickle_file: 'str | None' = None, identification_threshold: 'float | None' = None)
 |      Constructor
 |      
 |      :param the_raw_results: object with the results of the estimation.
 |          Default: None.
 |      :type the_raw_results: biogeme.results.RawResults
 |      :param pickle_file: name of the file containing the raw results in
 |          pickle format. It can be a URL. Default: None.
 |      :type pickle_file: string
 |      :param identification_threshold: if the smallest eigenvalue of
 |          the second derivative matrix is lesser or equal to this
 |          parameter, the model is conside

We first display some summary information:

In [18]:
print(results.print_general_statistics())


Number of estimated parameters:	1
Sample size:	228
Excluded observations:	1511
Init log likelihood:	-148.3468
Final log likelihood:	-148.3468
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00674
Akaike Information Criterion:	298.6937
Bayesian Information Criterion:	302.123
Final gradient norm:	2.1508E-03
Nbr of threads:	8



Then we display the estimation results

In [19]:
print(results.get_estimated_parameters())


            Value  Rob. Std err  Rob. t-test  Rob. p-value
asc_car  0.595942      0.138376     4.306685      0.000017


The results are also available in an HTML file than can be opened in
your preferred browser.

In [20]:
print(f'HTML file: {results.data.htmlFileName}')


HTML file: binary_netherlands~00.html


You now need to improve the model by including attributes: travel cost and travel time.
1. Try a specification where the coefficients of these attributes are generic.
2. Try a specification where the coefficients of these attributes are alternative specific.
3. Try a specification where the time coefficient is generic and the cost coefficient is alternative specific.
4. Try a specification where the cost coefficient is generic and the time coefficient is alternative specific.
5. Comment the results. Identify your preferred model, and explain why.